#### Guiding Questions:
Who are the winners of the D1 division in the Germany Football Association (Bundesliga) in the last decade?

Which teams have been relegated in the past 10 years?

Does Octoberfest affect the performance of Bundesliga?

Which season of Bundesliga was the most competitive in the last decade?

What's the best month to watch Bundesliga?

In [ ]:
import findspark
findspark.init()
import pyspark


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import instr,col,column, expr, coalesce, broadcast
from pyspark.sql.functions import count, first, last,min, max, sum, avg, sumDistinct,sum_distinct, pow, desc, countDistinct, approx_count_distinct, dense_rank, rank
from pyspark.sql.functions import corr, round, bround, monotonically_increasing_id, mean, format_number, var_pop, stddev_pop,var_samp, stddev_samp, skewness, kurtosis, covar_pop, covar_samp
from pyspark.sql.functions import initcap , lower, upper, lit, ltrim, rtrim, rpad, lpad, trim, regexp_replace, regexp_extract, translate, collect_set, collect_list
from pyspark.sql.functions import when,current_date, current_timestamp, date_add, date_sub, datediff, to_date, months_between, dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
from pyspark.sql.window import Window

In [ ]:
df_matches  = spark.read.csv('data/Matches.csv',inferSchema=True,header=True)
df_teams  = spark.read.csv('data/Teams.csv',inferSchema=True,header=True)
df_team_matches  = spark.read.csv('data/Teams_in_Matches.csv',inferSchema=True,header=True)
df_team_id  = spark.read.csv('data/Unique_Teams.csv',inferSchema=True,header=True)

In [ ]:
df_matches.show(5)
df_teams.select('*').show(5)
df_team_matches.show(5)
df_team_id.show(5)

In [ ]:
df1=df_matches.select(col('Season'),col('HomeTeam').alias('Team'),col('Div'))\
    .union(df_matches.select(col('Season'),col('AwayTeam').alias('Team'),col('Div')))

In [ ]:
#df2=df1.groupby('HomeTeam','Season','Div').agg(count('*')).orderBy(col('HomeTeam').desc(),col('Season').desc(),col('Div').desc())
df2=df1.groupby('Team','Season','Div').agg(count('*')).orderBy('Team','Season', ascending=False)

In [ ]:
df3=df2.alias('a').join(df2.alias('b'),(col('a.Team')==col('b.Team'))&(col('a.Season')>col('b.Season'))&(col('a.Div')>col('b.Div')))
df4=df3.select('a.Team').distinct()

In [ ]:
df4.show()